Optional A for the Project of DLAI Fall semester 2019/2020.

Authors:
*   Roser Batlle Roca
*   Jordi Biosca Caro
*   María González i Calabuig
*   Carlos Alejandro López Molina


In [0]:
import torch

import numpy as np
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data.sampler import SubsetRandomSampler
from tabulate import tabulate

seed = 1
torch.manual_seed(seed)
np.random.seed(seed)

In [0]:
hparams = {
    'batch_size': 64,
    'num_epochs': 5,
    'test_batch_size': 64,
    'hidden_size': 64,
    'hidden_size_2': 128,
    'hidden_size_3': 64,
    'num_classes': 10,
    'num_inputs': 1, 
    'learning_rate': 1e-3,
    'log_interval': 100,
    'num_workers': 4,
    'kernel_size': 3,
    'encoder_key': [1, 5, 25, 50, 100]
}

hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
mnist_trainset = datasets.MNIST('data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
mnist_testset = datasets.MNIST('data', train=False, 
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

num_train = len(mnist_trainset)
indices = list(range(num_train))
test_size = 0.05
split = int(np.floor(test_size * num_train))

train_id, test_id = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_id)

test_sampler = SubsetRandomSampler(test_id)

train_loader = torch.utils.data.DataLoader(mnist_trainset, 
                                           hparams['batch_size'], 
                                           shuffle=False,
                                           sampler=train_sampler,
                                           num_workers=hparams['num_workers'], 
                                           )

test_loader = torch.utils.data.DataLoader(mnist_trainset, 
                                          hparams['batch_size'], 
                                          shuffle=False,
                                          sampler=test_sampler,
                                          num_workers=hparams['num_workers'], 
                                          )


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27203721.84it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


32768it [00:00, 452639.33it/s]
  1%|          | 16384/1648877 [00:00<00:11, 144128.54it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


1654784it [00:00, 7060670.30it/s]                            
8192it [00:00, 183695.75it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


![Gaussian Variationa Autoencoder](https://blog.bayeslabs.co/assets/img/vae-gaussian.png)

In [0]:
class VAE(nn.Module):

  def __init__(self):
    super(VAE,self).__init__()

    self.fc1m = nn.Linear(1280, 20)
    self.fc1s = nn.Linear(1280, 20)
    self.fc2 = nn.Linear(20, 400)
    self.fc3 = nn.Linear(400, 1280)

    self.encoder = nn.Sequential(
        nn.Conv2d(hparams['num_inputs'], hparams['hidden_size'], 4, padding=3, stride=2), # surten 16,16
        nn.ReLU(True),
        nn.Conv2d(hparams['hidden_size'], hparams['hidden_size_2'], 4, padding=1, stride=2), #surten 8,8
        nn.ReLU(True),
        nn.Conv2d(hparams['hidden_size_2'], 5, 4, stride=2), #surten 3,3
        nn.ReLU(True),
        nn.MaxPool2d(2, stride=1) # surten 2,2
    )
    

    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(5, hparams['hidden_size_2'], 3, stride=2), #surten 5,5
        nn.ReLU(True),
        nn.ConvTranspose2d(hparams['hidden_size_2'], hparams['hidden_size'], 5, stride=3, padding=1), # surten 15,15
        nn.ReLU(True),
        nn.ConvTranspose2d(hparams['hidden_size'], hparams['num_inputs'], 2, stride=2, padding=1), #surten 28,28
        nn.Tanh(),
    )
  def encode(self,x):
    x = self.encoder(x)
    x = x.view(-1)
    return  self.fc1m(x),self.fc1s(x)
    
  #Explicació a la imatge
  def reparametrize(self, mu, logvar):
    std = logvar.mul(0.5).exp_()
    if torch.cuda.is_available():
        eps = torch.cuda.FloatTensor(std.size()).normal_()
    else:
        eps = torch.FloatTensor(std.size()).normal_()
    eps = Variable(eps)
    return eps.mul(std).add_(mu)

  def forward(self,x):
    mu, logvar = self.encode(x)
    x = self.reparametrize(mu, logvar)
    x = self.fc3(self.fc2(x))
    #Falta transformar els 1280 a vector o canviar el decoder
    x = self.decoder(x)
    return x, mu, logvar

In [0]:
# Trainning process

model = VAE()
model = model.to(hparams['device'])
distance = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'], weight_decay=1e-5)
losses_train = [] 
losses_test = []
for epoch in range (hparams['num_epochs']):
  for batch_id,data in enumerate(train_loader):
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_train.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
  for data in test_loader:
    img, _ = data
    img = img.to(hparams['device'])
    # Forward pass
    output = model(img)
    loss = distance(output, img)
    losses_test.append(loss.cpu().data.item())
    # Backward pass
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

RuntimeError: ignored